In [1]:
import sys        
import os         
import pygame      
import numpy as np
import math
import time
import random

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()
BG_COLOR = (0, 0, 0)
screen = pygame.display.set_mode((600, 500), pygame.RESIZABLE)
SCREEN_SIZE = screen.get_size()


class Game:
    def  __init__(self, screen):     
        self.screen = screen         
        self.starship = Starship() 
        self.bullets = []           
        self.asteroids = []
        self.mouse_pressed = False
        self.fire_rate = 0.2 

    def handle_events(self, frame):
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT: 
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                self.mouse_pressed = True
            if event.type == pygame.MOUSEBUTTONUP:
                self.mouse_pressed = False
        if self.mouse_pressed and (time.time() - self.starship.last_bullet_time) > self.fire_rate:
            new_bullet = self.starship.fire()     # Bullet(self.pos)
            self.bullets.append(new_bullet)
        if frame % 50 == 0:
            self.asteroids.append(Asteroid())

    def move_objects(self, objects_list):    
        for obj_idx, _ in enumerate(objects_list): 
            for o_idx, _ in enumerate(objects_list[obj_idx]): 
                objects_list[obj_idx][o_idx].move()
                
    def check_collisions(self):
        asteroids_rects = []
        for asteroid in self.asteroids:
            asteroids_rects.append(asteroid.rect)
        #### Asteroids vs bullets
        for idx, bullet in enumerate(self.bullets):
            hit = bullet.rect.collidelist(asteroids_rects)
            if hit != -1:
                del self.asteroids[hit]
                del self.bullets[idx]
        ### Bullets vs borders
        for idx, bullet in enumerate(self.bullets):
            if bullet.pos[0] > SCREEN_SIZE[0] or bullet.pos[1] > SCREEN_SIZE[1] or bullet.pos[0] < 0 or bullet.pos[1] < 0:
                del self.bullets[idx]
        ### Starship vs Asteroids
        hit = self.starship.rect.collidelist(asteroids_rects)
        if hit != -1:
            sys.exit()


    def draw(self, objects_list):
        screen.fill(BG_COLOR)
        for objects in objects_list:
            for obj in objects:
                self.screen.blit(obj.image, obj.rect)
        pygame.display.update()

    def run(self):
        clock = pygame.time.Clock()
        frame = 0
        while True:
            clock.tick(60)
            self.handle_events(frame)
            self.check_collisions()
            self.move_objects([[self.starship], self.bullets, self.asteroids])
            self.draw([[self.starship], self.bullets, self.asteroids])
            frame += 1

class Starship:
    def __init__(self):
        self.pos = np.array([SCREEN_SIZE[0] / 2, SCREEN_SIZE[1] / 2])
        self.original_image = pygame.image.load(os.path.join("images", "starship.png"))
        self.image = self.original_image
        self.rect = self.image.get_rect(center=self.pos)
        self.last_bullet_time = 0

    def move(self):                    
        mouse_pos = pygame.mouse.get_pos()   
        direction = mouse_pos - self.pos     
        angle = self.calculate_angle(mouse_pos) 
        self.speed = direction / 40         
        self.pos += self.speed              
        self.image = pygame.transform.rotate(self.original_image, int(angle))
        self.rect = self.image.get_rect(center=self.pos) 

    def calculate_angle(self, mouse_pos):    
        rel_x, rel_y = mouse_pos - self.pos  
        angle = (180 / math.pi) * -math.atan2(rel_y, rel_x) + 90 
        return angle
    
    def fire(self):
        self.last_bullet_time = time.time()
        return Bullet(self.pos.copy())
    

class Bullet:
    original_image = pygame.image.load(os.path.join("images", "bullet.png"))
    
    def __init__(self, pos):
        self.pos = pos
        mouse_pos = pygame.mouse.get_pos()
        self.direction = mouse_pos - pos
        self.speed = self.direction / max(abs(self.direction)) * 10
        self.angle = self.calculate_angle(mouse_pos)
        self.image = pygame.transform.rotate(self.original_image, int(self.angle))
        self.rect = self.image.get_rect(center=self.pos)
        
    def calculate_angle(self, mouse_pos):    
        rel_x, rel_y = mouse_pos - self.pos  
        angle = (180 / math.pi) * -math.atan2(rel_y, rel_x) + 90 
        return angle
    
    def move(self):
        self.pos += self.speed
        self.rect = self.image.get_rect(center=self.pos)

class Asteroid:
    original_images = [pygame.image.load(os.path.join("images", "ast1_medium.png")),
                       pygame.image.load(os.path.join("images", "ast2_medium.png")),
                       pygame.image.load(os.path.join("images", "ast3_medium.png")),
                       pygame.image.load(os.path.join("images", "ast4_medium.png"))]
    def __init__(self):
        self.image = random.choice(self.original_images)
        self.pos = random.choice([self.left_pos, self.right_pos, self.top_pos, self.bottom_pos])()
        self.direction = pygame.mouse.get_pos() - self.pos
        self.speed = self.direction / 250
        self.rect = self.image.get_rect(center=self.pos)
        
    def left_pos(self):
        return np.array((-150, random.uniform(0, SCREEN_SIZE[1])))
    
    def right_pos(self):
        return np.array((SCREEN_SIZE[0] + 150, random.uniform(0, SCREEN_SIZE[1])))
    
    def top_pos(self):
        return np.array((random.uniform(0, SCREEN_SIZE[0]), -150))
    
    def bottom_pos(self):
        return np.array((random.uniform(0, SCREEN_SIZE[0]), SCREEN_SIZE[1] + 150))
    
    def move(self):
        self.check_borders()
        self.pos += self.speed
        self.rect = self.image.get_rect(center=self.pos)
        
    def check_borders(self):
        if self.pos[0] > (SCREEN_SIZE[0] + 150):
            self.pos[0] = -150
        elif (self.pos[0] + 150) < 0:
            self.pos[0] = SCREEN_SIZE[0] + 150
        elif (self.pos[1] - 150) > SCREEN_SIZE[1]:
            self.pos[1] = -150
        elif (self.pos[1] + 150) < 0:
            self.pos[1] = SCREEN_SIZE[1] + 150
        self.rect = self.image.get_rect(center=self.pos, width=10, height=10)
        
game = Game(screen)
game.run()


SystemExit: 

/home/roman/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
a = time.time()

In [5]:
time.time() - a

6.165595293045044

In [9]:
random.choice([1, 2, 3, 4])

4

In [15]:
random.uniform(-2, 2)

-0.13138453564932684